In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
image_array = cv2.imread('AffectNet/train/0/image0000006.jpg', cv2.IMREAD_GRAYSCALE)

In [ ]:
image_array.shape

In [ ]:
plt.imshow(image_array)

In [ ]:
DATA_DIR = 'AffectNet/'
CATEGORIES = ['train', 'val', 'test']

In [ ]:
CLASSES = ['0', '1', '2', '3', '4', '5', '6', '7']

In [ ]:
for categoy in CATEGORIES:
    for class_name in CLASSES:
        path = os.path.join(DATA_DIR, categoy, class_name)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img))
            plt.imshow(img_array, cmap='gray')
            plt.show()
            break
        break
    break

In [ ]:
image_size = 224
image_array = cv2.resize(image_array, (image_size, image_size))
plt.imshow(image_array, cmap='gray')
plt.show()

In [ ]:
image_array.shape

# Read all the images and convertin them to array

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def data_generator(data_dir, image_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_gen = datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=image_size,
        batch_size=batch_size,
        class_mode='sparse',
    )

    val_gen = datagen.flow_from_directory(
        os.path.join(data_dir, 'val'),
        target_size=image_size,
        batch_size=batch_size,
        class_mode='sparse',
    )

    test_gen = datagen.flow_from_directory(
        os.path.join(data_dir, 'test'),
        target_size=image_size,
        batch_size=batch_size,
        class_mode='sparse'
    )

    return train_gen, val_gen, test_gen

In [ ]:
image_size = (224, 224)

In [ ]:
batch_size = 32

In [ ]:
train_gen, val_gen, test_gen = data_generator(DATA_DIR, image_size=image_size, batch_size=batch_size)

In [ ]:
test_gen.class_indices

# Deep learning model for training - Transfer Learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Activation # type: ignore
from tensorflow.keras.models import Model  # type: ignore

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
%pip install tensorflow-gpu

In [ ]:
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
base_input = model.input

In [ ]:
base_input

In [ ]:
base_output = model.layers[-2].output

In [ ]:
x = base_output
x = GlobalAveragePooling2D()(x)
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

In [ ]:
new_model = Model(inputs=base_input, outputs=predictions)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=0.0001))

In [ ]:
# Huấn luyện mô hình
new_model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    validation_data=val_gen,
    validation_steps=val_gen.samples // batch_size,
    epochs=100
)


In [ ]:
# save the model
new_model.save('emotion_model.h5')